# Group Assignment - Surviving the Analysis - Gr-54

## EDA

1. Separate data into each component failure data type and cencored data
2. Check for impact of env components - Carpets, Pets, usage Patterns
3. Check for obs which had failure but none of the 3 components failed (Censored)
4. More than one part failed when sent for repair. Check how to handle it
5. Check hours of use and impact on the sensors
   1. about 30% of the dataset had pets
   2. 30% of those who had pets had failures
   3. for the ones with battery failures, 50% had pets
   4. 30% had pets for ir failures
   5. however, over 87% of impact failures happened to those with pets. This makes sense cuz pets keep on bumping into shit.
   6. Battery failure increases (30%) when there are pets. Probably cuz they shed.
6. Check if having pets increases the time the robo runs. Could explain the battery part.
7. Check impact of carpet and pets on failures

### Assumptions:

1. Survival of each component is independent of the other
2. if one part fails, the others that did not fail go into censored data??
3. Only one part fails at a time. the 2 rows where 2 parts failed have to be handled.

## Modelling

1. Do 4 models - Exponential, weibull, kaplan-mayer, and Cox
   1. Start with a very simple model - consider dec 31 as the censoring time for everything
   2. Next iteration, consider censoring time for component x as failure time of component y
2. Do all forumals, assumptions, all dists n all
3. Handling the changing censoring time
   1. If we use models that consider constant censoring time
      1. Filter out all other failures
   2. Using models that can handle changing survival time,
      1. separate them out
      2. Set the failure time of component 1 as censoring time of the other components

## Inference

1. Check the info provided for confidence interval

#### Members:<br>

Boglarka ACS<br>
Chavisha Arora<br>
Sander Bergman<br>
Aniket Mishra<br>


## Random thoughts to consider:

.


In [1]:
import pandas as pd
import re
from varname import nameof

import warnings

warnings.filterwarnings("ignore")

In [2]:
def sanitize(input_str: str) -> str:
    return re.sub(r"[^A-Za-z0-9]+", "_", input_str)

In [3]:
df = pd.read_csv("../data/DirtSlurper3000.csv", skiprows=13)
df.head(2)

,Registration date,Total usage time,Pets,Carpet score,Sent for repair,Failure date,Battery status,Impact status,IR status
0,29Jul2019,280,1,1,NO,---,---,---,---
1,8Sep2019,89,0,1,NO,---,---,---,---


In [4]:
df.describe(include="all").T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Registration date,5387,1742,14May2018,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Total usage time,5387.0,NaN,NaN,NaN,1050.237609,760.877879,0.0,436.0,912.0,1498.5,4019.0
Pets,5387.0,NaN,NaN,NaN,0.343234,0.474833,0.0,0.0,0.0,1.0,1.0
Carpet score,5387.0,NaN,NaN,NaN,2.709115,1.344708,1.0,2.0,2.0,3.0,8.0
Sent for repair,5387,2,NO,4166,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Failure date,5387,712,---,4166,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Battery status,5387,3,---,4166,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Impact status,5387,3,---,4166,NaN,NaN,NaN,NaN,NaN,NaN,NaN
IR status,5387,3,---,4166,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5387 entries, 0 to 5386
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Registration date  5387 non-null   object
 1   Total usage time   5387 non-null   int64 
 2   Pets               5387 non-null   int64 
 3   Carpet score       5387 non-null   int64 
 4   Sent for repair    5387 non-null   object
 5   Failure date       5387 non-null   object
 6   Battery status     5387 non-null   object
 7   Impact status      5387 non-null   object
 8   IR status          5387 non-null   object
dtypes: int64(3), object(6)
memory usage: 1.8 MB


In [6]:
df.columns = [sanitize(col).lower() for col in df.columns]
df.head(2)

,registration_date,total_usage_time,pets,carpet_score,sent_for_repair,failure_date,battery_status,impact_status,ir_status
0,29Jul2019,280,1,1,NO,---,---,---,---
1,8Sep2019,89,0,1,NO,---,---,---,---


In [7]:
df.loc[df["failure_date"] == "---", "failure_date"] = "1Jan1970"
df["registration_date"] = pd.to_datetime(df["registration_date"])
df["failure_date"] = pd.to_datetime(df["failure_date"])

In [8]:
df = df.sort_values(["registration_date"]).reset_index(drop=True)

In [9]:
df.head(1)

,registration_date,total_usage_time,pets,carpet_score,sent_for_repair,failure_date,battery_status,impact_status,ir_status
0,2015-01-01,262,0,1,YES,2015-10-23,OK,OK,Damage


In [10]:
for col in df.columns:
    all_unique = sorted(df[col].unique())
    len_unique = len(all_unique)
    if len_unique < 10:
        print(col)
        print(all_unique)
        print()

pets
[np.int64(0), np.int64(1)]

carpet_score
[np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8)]

sent_for_repair
['NO', 'YES']

battery_status
['---', 'Damage', 'OK']

impact_status
['---', 'Damage', 'OK']

ir_status
['---', 'Damage', 'OK']



In [11]:
replace_values_with_int = {"---": 0, "Damage": 1, "OK": 2, "NO": 0, "YES": 1}

In [12]:
df[["sent_for_repair", "battery_status", "impact_status", "ir_status"]] = df[
    ["sent_for_repair", "battery_status", "impact_status", "ir_status"]
].replace(replace_values_with_int)

In [13]:
df["had_failure"] = 0
df["time_to_failure"] = None
# df["time_to_failure_battery_status"] = None
# df["time_to_failure_impact_status"] = None
# df["time_to_failure_ir_status"] = None
df["type_of_failure"] = "Censored"

In [14]:
df.loc[df["failure_date"] != "1970-01-01", "had_failure"] = 1
mask = df["had_failure"] == 1
df.loc[mask, "time_to_failure"] = (df["failure_date"] - df["registration_date"]).dt.days
df["time_to_failure"].fillna(-1, inplace=True)

In [15]:
df.loc[((df["had_failure"] == 1) & (df["battery_status"] == 1)), "type_of_failure"] = (
    "Battery"
)
df.loc[((df["had_failure"] == 1) & (df["impact_status"] == 1)), "type_of_failure"] = (
    "Impact"
)
df.loc[((df["had_failure"] == 1) & (df["ir_status"] == 1)), "type_of_failure"] = "Ir"

df.loc[
    ((df["had_failure"] == 1) & (df["ir_status"] == 1) & (df["battery_status"] == 1)),
    "type_of_failure",
] = "Battery_and_Ir"
df.loc[
    ((df["had_failure"] == 1) & (df["ir_status"] == 1) & (df["impact_status"] == 1)),
    "type_of_failure",
] = "Ir_and_Impact"
df.loc[
    (
        (df["had_failure"] == 1)
        & (df["battery_status"] == 1)
        & (df["impact_status"] == 1)
    ),
    "type_of_failure",
] = "Battery_and_Impact"
df.loc[
    ((df["had_failure"] == 1) & (df["type_of_failure"] == "Censored")),
    "type_of_failure",
] = "Unknown_Component"

In [16]:
df.loc[df["type_of_failure"] == "Unknown_Component"].shape

(39, 12)

## Sanity Checks - EDA


### Checking if we missed to fill any failures


In [17]:
df.loc[df["type_of_failure"].isna()]

,registration_date,total_usage_time,pets,carpet_score,sent_for_repair,failure_date,battery_status,impact_status,ir_status,had_failure,time_to_failure,type_of_failure


In [18]:
df.loc[df["type_of_failure"].str.contains("and")]

,registration_date,total_usage_time,pets,carpet_score,sent_for_repair,failure_date,battery_status,impact_status,ir_status,had_failure,time_to_failure,type_of_failure
2,2015-01-01,2454,1,4,1,2018-11-28,1,1,2,1,1427,Battery_and_Impact
636,2015-08-03,2966,1,3,1,2017-12-25,1,1,2,1,875,Battery_and_Impact


### Checking for Had a failure but not sent for repair


In [19]:
df.loc[((df["sent_for_repair"] == 0) & (df["had_failure"] == 1))]

,registration_date,total_usage_time,pets,carpet_score,sent_for_repair,failure_date,battery_status,impact_status,ir_status,had_failure,time_to_failure,type_of_failure


### Multiple components failed at the same time

## Context:

if battery is below 80% (Not a failure) but is sent to repair cuz another component failed.<br>
Manufacturer guarantees 2400 hours at 80%, so this can be considered a failure. tho people will not notice this.

i.e.<br> If battery is marked as failed along with other components, it means the health of the battry was below 80% when it went to get fixed due to the other sensor issue. We might wanna account for that. So whenever we see battery_and_othersensor, we consider both as failed and failure time as the failure time for both components.

#### Reason: Contractual - Mentioned in documentation.


In [20]:
df.loc[
    ((df["battery_status"] == 1) & (df["impact_status"] == 1))
    | ((df["battery_status"] == 1) & (df["ir_status"] == 1))
    | ((df["ir_status"] == 1) & (df["impact_status"] == 1))
]

,registration_date,total_usage_time,pets,carpet_score,sent_for_repair,failure_date,battery_status,impact_status,ir_status,had_failure,time_to_failure,type_of_failure
2,2015-01-01,2454,1,4,1,2018-11-28,1,1,2,1,1427,Battery_and_Impact
636,2015-08-03,2966,1,3,1,2017-12-25,1,1,2,1,875,Battery_and_Impact


### Failure on one device but other 2 devices are censored (0) and not No failure (2)


In [21]:
df.loc[
    (
        (df["battery_status"] == 1)
        & ((df["impact_status"] == 0) | (df["ir_status"] == 0))
    )
    | (
        (df["impact_status"] == 1)
        & ((df["battery_status"] == 0) | (df["ir_status"] == 0))
    )
    | (
        (df["ir_status"] == 1)
        & ((df["impact_status"] == 0) | (df["battery_status"] == 0))
    )
]

,registration_date,total_usage_time,pets,carpet_score,sent_for_repair,failure_date,battery_status,impact_status,ir_status,had_failure,time_to_failure,type_of_failure


### Had failure but all 3 components were marked as not failed (Unknown component failure)


In [22]:
df.loc[
    (df["had_failure"] == 1)
    & (df["battery_status"] == 2)
    & (df["impact_status"] == 2)
    & (df["ir_status"] == 2)
].shape

(39, 12)

### Context:

We may consider the above cases where a part other than the other 3 failed.<br>
We could use the failure time as censoring time for the 3 components.


### Had failure but all 3 components were censored (0)


In [23]:
df.loc[
    (df["had_failure"] == 1)
    & (df["battery_status"] == 0)
    & (df["impact_status"] == 0)
    & (df["ir_status"] == 0)
].shape

(0, 12)

## Separating the dataset into different failures (No longer needed)


In [24]:
df_battery_failure = df.loc[df["battery_status"] == 1]
df_impact_failure = df.loc[df["impact_status"] == 1]
df_ir_failure = df.loc[df["ir_status"] == 1]

### Checking info on all component failures

#### Battery Failure


In [25]:
df_battery_failure.describe(include="all").T

,count,unique,top,freq,mean,min,25%,50%,75%,max,std
registration_date,826,NaN,NaN,NaN,2016-01-12 18:21:47.506053376,2015-01-01 00:00:00,2015-05-24 06:00:00,2015-11-01 12:00:00,2016-07-11 18:00:00,2018-09-30 00:00:00,NaN
total_usage_time,826.0,NaN,NaN,NaN,1622.473366,40.0,1101.25,1522.0,2086.0,3731.0,699.294633
pets,826.0,NaN,NaN,NaN,0.487893,0.0,0.0,0.0,1.0,1.0,0.500156
carpet_score,826.0,NaN,NaN,NaN,3.002421,1.0,2.0,3.0,4.0,8.0,1.402161
sent_for_repair,826.0,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,0.0
failure_date,826,NaN,NaN,NaN,2018-12-19 18:28:45.907990272,2016-02-15 00:00:00,2018-07-05 06:00:00,2019-02-16 12:00:00,2019-08-04 18:00:00,2019-12-30 00:00:00,NaN
battery_status,826.0,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,0.0
impact_status,826.0,NaN,NaN,NaN,1.997579,1.0,2.0,2.0,2.0,2.0,0.049177
ir_status,826.0,NaN,NaN,NaN,2.0,2.0,2.0,2.0,2.0,2.0,0.0
had_failure,826.0,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,0.0


In [26]:
df_battery_failure.info()

<class 'pandas.core.frame.DataFrame'>
Index: 826 entries, 2 to 3990
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   registration_date  826 non-null    datetime64[ns]
 1   total_usage_time   826 non-null    int64         
 2   pets               826 non-null    int64         
 3   carpet_score       826 non-null    int64         
 4   sent_for_repair    826 non-null    int64         
 5   failure_date       826 non-null    datetime64[ns]
 6   battery_status     826 non-null    int64         
 7   impact_status      826 non-null    int64         
 8   ir_status          826 non-null    int64         
 9   had_failure        826 non-null    int64         
 10  time_to_failure    826 non-null    int64         
 11  type_of_failure    826 non-null    object        
dtypes: datetime64[ns](2), int64(9), object(1)
memory usage: 83.9+ KB


#### Impact Failure


In [27]:
df_impact_failure.describe(include="all").T

,count,unique,top,freq,mean,min,25%,50%,75%,max,std
registration_date,130,NaN,NaN,NaN,2015-12-10 23:26:46.153846272,2015-01-01 00:00:00,2015-05-12 12:00:00,2015-09-25 12:00:00,2016-05-24 12:00:00,2018-01-14 00:00:00,NaN
total_usage_time,130.0,NaN,NaN,NaN,2686.769231,1278.0,2356.75,2752.0,2991.0,4019.0,486.255931
pets,130.0,NaN,NaN,NaN,0.876923,0.0,1.0,1.0,1.0,1.0,0.329796
carpet_score,130.0,NaN,NaN,NaN,1.992308,1.0,1.0,2.0,3.0,5.0,0.976437
sent_for_repair,130.0,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,0.0
failure_date,130,NaN,NaN,NaN,2019-02-28 16:36:55.384615424,2016-09-30 00:00:00,2018-10-08 00:00:00,2019-04-28 12:00:00,2019-09-26 06:00:00,2019-12-30 00:00:00,NaN
battery_status,130.0,NaN,NaN,NaN,1.984615,1.0,2.0,2.0,2.0,2.0,0.123553
impact_status,130.0,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,0.0
ir_status,130.0,NaN,NaN,NaN,2.0,2.0,2.0,2.0,2.0,2.0,0.0
had_failure,130.0,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,0.0


In [28]:
df_impact_failure.info()

<class 'pandas.core.frame.DataFrame'>
Index: 130 entries, 2 to 3231
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   registration_date  130 non-null    datetime64[ns]
 1   total_usage_time   130 non-null    int64         
 2   pets               130 non-null    int64         
 3   carpet_score       130 non-null    int64         
 4   sent_for_repair    130 non-null    int64         
 5   failure_date       130 non-null    datetime64[ns]
 6   battery_status     130 non-null    int64         
 7   impact_status      130 non-null    int64         
 8   ir_status          130 non-null    int64         
 9   had_failure        130 non-null    int64         
 10  time_to_failure    130 non-null    int64         
 11  type_of_failure    130 non-null    object        
dtypes: datetime64[ns](2), int64(9), object(1)
memory usage: 13.2+ KB


#### IR Failure


In [29]:
df_ir_failure.describe(include="all").T

,count,unique,top,freq,mean,min,25%,50%,75%,max,std
registration_date,228,NaN,NaN,NaN,2016-09-06 00:31:34.736841984,2015-01-01 00:00:00,2015-09-26 00:00:00,2016-06-20 00:00:00,2017-08-09 06:00:00,2019-06-16 00:00:00,NaN
total_usage_time,228.0,NaN,NaN,NaN,663.77193,10.0,251.5,480.5,949.0,3270.0,565.173468
pets,228.0,NaN,NaN,NaN,0.333333,0.0,0.0,0.0,1.0,1.0,0.472442
carpet_score,228.0,NaN,NaN,NaN,2.754386,1.0,2.0,3.0,4.0,7.0,1.43623
sent_for_repair,228.0,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,0.0
failure_date,228,NaN,NaN,NaN,2018-01-21 21:41:03.157894656,2015-06-27 00:00:00,2017-02-11 06:00:00,2018-02-12 12:00:00,2019-02-22 00:00:00,2019-12-25 00:00:00,NaN
battery_status,228.0,NaN,NaN,NaN,2.0,2.0,2.0,2.0,2.0,2.0,0.0
impact_status,228.0,NaN,NaN,NaN,2.0,2.0,2.0,2.0,2.0,2.0,0.0
ir_status,228.0,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,0.0
had_failure,228.0,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,0.0


In [30]:
df_ir_failure.info()

<class 'pandas.core.frame.DataFrame'>
Index: 228 entries, 0 to 4752
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   registration_date  228 non-null    datetime64[ns]
 1   total_usage_time   228 non-null    int64         
 2   pets               228 non-null    int64         
 3   carpet_score       228 non-null    int64         
 4   sent_for_repair    228 non-null    int64         
 5   failure_date       228 non-null    datetime64[ns]
 6   battery_status     228 non-null    int64         
 7   impact_status      228 non-null    int64         
 8   ir_status          228 non-null    int64         
 9   had_failure        228 non-null    int64         
 10  time_to_failure    228 non-null    int64         
 11  type_of_failure    228 non-null    object        
dtypes: datetime64[ns](2), int64(9), object(1)
memory usage: 23.2+ KB


#### Failure percentages


In [31]:
total_failure_percentage = (df.loc[df["had_failure"] == 1].shape[0] / df.shape[0]) * 100
battery_failure_percentage = (df_battery_failure.shape[0] / df.shape[0]) * 100
impact_failure_percentage = (df_impact_failure.shape[0] / df.shape[0]) * 100
ir_failure_percentage = (df_ir_failure.shape[0] / df.shape[0]) * 100

total_rows = df.shape[0]
print(f"{nameof(total_rows)}: {total_rows}")
var_dict = {
    "Total Failure": [
        {nameof(total_failure_percentage): total_failure_percentage},
        df.loc[df["had_failure"] == 1].shape[0],
        df["registration_date"].min(),
        df["registration_date"].max(),
    ],
    nameof(df_battery_failure): [
        {nameof(battery_failure_percentage): battery_failure_percentage},
        df_battery_failure.shape[0],
        df_battery_failure["registration_date"].min(),
        df_battery_failure["registration_date"].max(),
    ],
    nameof(df_impact_failure): [
        {nameof(impact_failure_percentage): impact_failure_percentage},
        df_impact_failure.shape[0],
        df_impact_failure["registration_date"].min(),
        df_impact_failure["registration_date"].max(),
    ],
    nameof(df_ir_failure): [
        {nameof(ir_failure_percentage): ir_failure_percentage},
        df_ir_failure.shape[0],
        df_ir_failure["registration_date"].min(),
        df_ir_failure["registration_date"].max(),
    ],
}

for key, percent in var_dict.items():
    print(
        f"\n{key}\n{percent[0]}, \nRows: {percent[1]}, \nMin Time:{percent[2]}, \nMax Time:{percent[3]}\n"
    )

total_rows: 5387

Total Failure
{'total_failure_percentage': 22.665676628921478}, 
Rows: 1221, 
Min Time:2015-01-01 00:00:00, 
Max Time:2019-12-31 00:00:00


df_battery_failure
{'battery_failure_percentage': 15.333209578615184}, 
Rows: 826, 
Min Time:2015-01-01 00:00:00, 
Max Time:2018-09-30 00:00:00


df_impact_failure
{'impact_failure_percentage': 2.4132170038982736}, 
Rows: 130, 
Min Time:2015-01-01 00:00:00, 
Max Time:2018-01-14 00:00:00


df_ir_failure
{'ir_failure_percentage': 4.232411360683126}, 
Rows: 228, 
Min Time:2015-01-01 00:00:00, 
Max Time:2019-06-16 00:00:00



## Getting Censored Data - No Failures at all


In [32]:
df_censored = df.loc[df["had_failure"] == 0]

In [33]:
df_censored

,registration_date,total_usage_time,pets,carpet_score,sent_for_repair,failure_date,battery_status,impact_status,ir_status,had_failure,time_to_failure,type_of_failure
3,2015-01-01,987,0,2,0,1970-01-01,0,0,0,0,-1,Censored
13,2015-01-05,727,0,3,0,1970-01-01,0,0,0,0,-1,Censored
14,2015-01-05,1850,0,2,0,1970-01-01,0,0,0,0,-1,Censored
15,2015-01-05,2393,0,2,0,1970-01-01,0,0,0,0,-1,Censored
21,2015-01-07,1530,0,3,0,1970-01-01,0,0,0,0,-1,Censored
...,...,...,...,...,...,...,...,...,...,...,...,...
5382,2019-12-30,1,0,5,0,1970-01-01,0,0,0,0,-1,Censored
5383,2019-12-30,1,0,1,0,1970-01-01,0,0,0,0,-1,Censored
5384,2019-12-31,0,1,1,0,1970-01-01,0,0,0,0,-1,Censored
5385,2019-12-31,0,0,3,0,1970-01-01,0,0,0,0,-1,Censored


# Impact of env factors on failures


In [34]:
result = (
    df_battery_failure.groupby("carpet_score")
    .agg(
        {
            "had_failure": "count",
        }
    )
    .reset_index()
)

result

,carpet_score,had_failure
0,1,104
1,2,236
2,3,224
3,4,138
4,5,80
5,6,30
6,7,12
7,8,2


In [35]:
result = (
    df_impact_failure.groupby("carpet_score")
    .agg(
        {
            "had_failure": "count",
        }
    )
    .reset_index()
)

result

,carpet_score,had_failure
0,1,48
1,2,47
2,3,25
3,4,8
4,5,2


In [36]:
result = (
    df_ir_failure.groupby("carpet_score").agg({"had_failure": "count"}).reset_index()
)

result

,carpet_score,had_failure
0,1,48
1,2,65
2,3,54
3,4,32
4,5,15
5,6,13
6,7,1


In [37]:
df["pets"].unique()

array([0, 1])

In [38]:
df_battery_failure.loc[df_battery_failure["pets"] == 1]

,registration_date,total_usage_time,pets,carpet_score,sent_for_repair,failure_date,battery_status,impact_status,ir_status,had_failure,time_to_failure,type_of_failure
2,2015-01-01,2454,1,4,1,2018-11-28,1,1,2,1,1427,Battery_and_Impact
6,2015-01-03,2190,1,1,1,2018-11-12,1,2,2,1,1409,Battery
19,2015-01-07,2676,1,2,1,2019-03-02,1,2,2,1,1515,Battery
24,2015-01-08,3022,1,1,1,2018-09-24,1,2,2,1,1355,Battery
25,2015-01-08,1302,1,3,1,2017-08-12,1,2,2,1,947,Battery
...,...,...,...,...,...,...,...,...,...,...,...,...
3458,2018-04-06,767,1,5,1,2019-02-10,1,2,2,1,310,Battery
3482,2018-04-14,963,1,3,1,2019-07-16,1,2,2,1,458,Battery
3726,2018-07-07,1012,1,2,1,2019-08-26,1,2,2,1,415,Battery
3794,2018-08-01,296,1,2,1,2019-10-09,1,2,2,1,434,Battery


In [39]:
df.loc[df["pets"] == 1].shape

(1849, 12)

In [40]:
df.loc[(df["pets"] == 1) & (df["had_failure"] == 1)].shape

(599, 12)

In [41]:
df_battery_failure.shape

(826, 12)

In [42]:
df.shape

(5387, 12)

In [43]:
df_battery_failure.shape

(826, 12)

In [44]:
print(df_ir_failure.loc[df_ir_failure["pets"] == 1].shape)
print(df_ir_failure.shape)

(76, 12)
(228, 12)


In [45]:
print(df_impact_failure.loc[df_impact_failure["pets"] == 1].shape)
print(df_impact_failure.shape)

(114, 12)
(130, 12)


In [46]:
114 / 130

0.8769230769230769

In [47]:
df.loc[(df["had_failure"] == 1) & (df["pets"] == 1)]["total_usage_time"].sum()

np.int64(1212167)

In [48]:
print(
    df_battery_failure.loc[
        (df_battery_failure["had_failure"] == 1) & (df_battery_failure["pets"] == 1)
    ]["total_usage_time"].sum()
)
print(
    df_battery_failure.loc[
        (df_battery_failure["had_failure"] == 1) & (df_battery_failure["pets"] == 0)
    ]["total_usage_time"].sum()
)
print(
    df_battery_failure.loc[(df_battery_failure["had_failure"] == 1)][
        "total_usage_time"
    ].sum()
)

815191
524972
1340163


In [49]:
print(
    df_ir_failure.loc[
        (df_ir_failure["had_failure"] == 1) & (df_ir_failure["pets"] == 1)
    ]["total_usage_time"].sum()
)
print(
    df_ir_failure.loc[
        (df_ir_failure["had_failure"] == 1) & (df_ir_failure["pets"] == 0)
    ]["total_usage_time"].sum()
)
print(df_ir_failure.loc[(df_ir_failure["had_failure"] == 1)]["total_usage_time"].sum())

71069
80271
151340


In [50]:
print(
    df_impact_failure.loc[
        (df_impact_failure["had_failure"] == 1) & (df_impact_failure["pets"] == 1)
    ]["total_usage_time"].sum()
)
print(
    df_impact_failure.loc[
        (df_impact_failure["had_failure"] == 1) & (df_impact_failure["pets"] == 0)
    ]["total_usage_time"].sum()
)
print(
    df_impact_failure.loc[(df_impact_failure["had_failure"] == 1)][
        "total_usage_time"
    ].sum()
)

print(
    df_impact_failure.loc[
        (df_impact_failure["had_failure"] == 1) & (df_impact_failure["pets"] == 1)
    ]["total_usage_time"].count()
)
print(
    df_impact_failure.loc[
        (df_impact_failure["had_failure"] == 1) & (df_impact_failure["pets"] == 0)
    ]["total_usage_time"].count()
)
print(
    df_impact_failure.loc[(df_impact_failure["had_failure"] == 1)][
        "total_usage_time"
    ].count()
)

315482
33798
349280
114
16
130


In [51]:
result = (
    df_impact_failure.loc[df_impact_failure["pets"] == 0]
    .groupby("carpet_score")
    .agg(
        {
            "had_failure": "count",
        }
    )
    .reset_index()
)

result

,carpet_score,had_failure
0,1,7
1,2,4
2,3,4
3,4,1


In [52]:
df.to_csv("../output/data_cleaned.csv", index=False)
df_battery_failure.to_csv("../output/df_battery_failure.csv", index=False)
df_impact_failure.to_csv("../output/df_impact_failure.csv", index=False)
df_ir_failure.to_csv("../output/df_ir_failure.csv", index=False)

In [53]:
df.loc[df["battery_status"] == 2, "battery_status"] = 0
df.loc[df["impact_status"] == 2, "impact_status"] = 0
df.loc[df["ir_status"] == 2, "ir_status"] = 0
df.to_csv("../output/cleaned_df_with_status_0_or_1.csv", index = False)